<a href="https://colab.research.google.com/github/LuciaManzanoUOC/MTP-UOC/blob/main/SkinPhototypeModifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skin Phototype modifier

Student: Manzano Gómez, Lucía Reyes

Supervisor: Rebrij, Romina Astrid

Date last modification: April 14th, 2022

## The Fitzpatrick skin type 

In this notebook we will develop code to modify images of skin and moles, in order to create images of phototypes IV, V and VI from images of phototypes I, II and III.

We use a dataset of 1,000 images of benign moles and 1,000 images of malignant melanomas of people with skin types I, II and III obtained in The International Skin Imaging Collaboration website [`ISIC archive`](https://www.isic-archive.com/). From these images, we created images of moles and melanomas of people with skin types IV, V, and VI.

In [1]:
# Import of libraries and use of GPUs to accelerate the training of the model.
%tensorflow_version 2.x
import tensorflow as tf
# Check if the GPUs are active
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))
  
import colorsys
from PIL import Image
from tensorflow.keras.preprocessing import image_dataset_from_directory
import re


Found GPU at: /device:GPU:0


In [3]:
# Mount our Google Drive where the database of images is stored.
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [4]:
# Once we are connected to Google Drive, unzip the database to obtain the
# images. These have already been divided among train (70%), test (15%) and 
# validation (15%).
!unzip drive/MyDrive/TFM/Databases/Database_to_modify.zip

Archive:  drive/MyDrive/TFM/Databases/Database_to_modify.zip
   creating: Database_to_modify/
   creating: Database_to_modify/test/
   creating: Database_to_modify/test/Benign/
  inflating: Database_to_modify/test/Benign/ISIC_4881109.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4881602.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4881841.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4882123.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4882333.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4883212.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4883407.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4883688.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4884189.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4884220.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4884394.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_4884879.jpeg  
  inflating: Database_to_modify/test/Benign/ISIC_488541

In [7]:
# We start preparing the data before working on the model.
# Definition of batch size.
BATCH_SIZE = 16
# Definition of images size.
IMG_SIZE = (224, 224)

# Build the train dataset.
train_dataset = image_dataset_from_directory("Database_to_modify/train",
                                             shuffle=False,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

Found 1400 files belonging to 2 classes.


In [8]:
# Build the test dataset.
test_dataset = image_dataset_from_directory("Database_to_modify/test",
                                             shuffle=False,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

Found 300 files belonging to 2 classes.


In [9]:
# Build the validation dataset.
val_dataset = image_dataset_from_directory("Database_to_modify/validation",
                                             shuffle=False,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

Found 300 files belonging to 2 classes.


In [10]:
# Function to modify images for darker phototypes
def modify_phototype(filepaths):
  for filename in filepaths:
    print("Editing phototype from image: ", filename)
    target_hue = 0

    image = Image.open(filename).convert('RGB')
    # RGB colors
    image.load()
    r, g, b = image.split()
    result_r, result_g, result_b = [], [], []
    # Process pixel by pixel
    for pixel_r, pixel_g, pixel_b in zip(r.getdata(), g.getdata(), b.getdata()):
        # We use HSV color to edit the hue, saturation and brightness of images
        h, s, v = colorsys.rgb_to_hsv(pixel_r / 255., pixel_b / 255., 
                                      pixel_g / 255.)
        v  = v - 0.25
        s = s + 0.3
        h = h - 0.2
        # Convert colors to RGB
        rgb = colorsys.hsv_to_rgb(target_hue, s, v)
        pixel_r, pixel_g, pixel_b = [int(x * 255.) for x in rgb]
        # Save pixel by pixel
        result_r.append(pixel_r)
        result_g.append(pixel_g)
        result_b.append(pixel_b)
    r.putdata(result_r)
    g.putdata(result_g)
    b.putdata(result_b)
    # Final image
    image = Image.merge('RGB', (r, g, b))
    # Overwrite the previous image
    image.save(filename)

In [11]:
modify_phototype(val_dataset.file_paths)

Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4809996.jpeg
Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4810500.jpeg
Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4811165.jpeg
Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4811211.jpeg
Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4811593.jpeg
Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4812480.jpeg
Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4812508.jpeg
Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4812889.jpeg
Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4813181.jpeg
Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4813681.jpeg
Editing phototype from image:  Database_to_modify/validation/Benign/ISIC_4813830.jpeg
Editing phototype from image:  Database_to_modify/vali

In [13]:
modify_phototype(test_dataset.file_paths)

Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4881109.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4881602.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4881841.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4882123.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4882333.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4883212.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4883407.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4883688.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4884189.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4884220.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4884394.jpeg
Editing phototype from image:  Database_to_modify/test/Benign/ISIC_4884879.jpeg
Editing phototype from image:  Database_

In [16]:
modify_phototype(train_dataset.file_paths)

Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4947778.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4947816.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4948341.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4948642.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4948722.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4949652.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4949672.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4949846.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4950321.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4950467.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4950948.jpeg
Editing phototype from image:  Database_to_modify/train/Benign/ISIC_4951035.jpeg
Editing phototype from image

In [19]:
!zip -r /content/Database_IV_V_VI.zip /content/Database_to_modify/

  adding: content/Database_to_modify/train/Malignant/ (stored 0%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0014557.jpeg (deflated 8%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0014171.jpeg (deflated 7%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0014832.jpeg (deflated 3%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0025512.jpeg (deflated 2%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0014372.jpeg (deflated 11%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0011922.jpeg (deflated 6%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0012151.jpeg (deflated 2%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0024898.jpeg (deflated 2%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0024367.jpeg (deflated 2%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0024552.jpeg (deflated 2%)
  adding: content/Database_to_modify/train/Malignant/ISIC_0011759.jpeg (deflated 4%

In [18]:
from google.colab import files
files.download("/content/Database_IV_V_VI.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>